<div dir="rtl">
<h1> توضیحات مربوط به هر cell در cell بالایی آن نوشته شده است.</h1>
</div>

<div dir="rtl">
<p>در خانه پایین کتابخانه pandas برای خواندن فایل و ساخت دیتافریم import شده است.</p>
</div>

In [56]:
import pandas as pd

<div dir="rtl">
<p>در خانه پایین فایل های دیتاست MNIST خوانده شده و به صورت dataframe در آمده اند.</p>
</div>

In [55]:
df_train = pd.read_csv("mnist_dataset/mnist_train.csv")
df_test = pd.read_csv("mnist_dataset/mnist_test.csv")

<div dir="rtl">
<p>خانه پایین ۵ ردیف اول train dataframe را نشان می دهد.</p>
</div>

In [26]:
df_train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<div dir="rtl">
<p>خانه پایین ۵ ردیف اول test dataframe را نشان می دهد.</p>
</div>

In [27]:
df_test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<div dir="rtl">
<p>کتابخانه هایی که قرار است از آن ها استفاده کنیم را در این خانه import می کنیم.</p>
</div>

In [59]:
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import optimizer

<div dir="rtl">
<p>من کارت گرافیک nvidia دارم و کتابخانه های مربوط به CUDA را نیز نصب کرده ام. اگر torch کارت گرافیک را تشخیص دهد، از آن برای پردازش استفاده می کنم.</p>
</div>

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

<div dir="rtl">
<p>در این خانه hyperparameter ها را تعریف کرده ام که توضیحات هر کدام در مقابلش نوشته شده است.</p>
</div>

In [61]:
input_size = 28*28  #Size of image
num_classes = 10  #the image number are in range 0-10
num_epochs = 10 #one cycle through the full train data
batch_size = 100 #sample size consider before updating the model’s weights
learning_rate = 0.001  #step size to update parameter
momentum = 0.01 #value of momentum

<div dir="rtl">
<p>در این خانه train dataframe را به numpy ndarray و سپس به torch tensor dataset و در انتها به torch data loader تبدیل کردم.</p>
</div>

In [31]:
X_train = df_train.drop("label", axis=1).to_numpy(dtype=np.float32)
y_train = df_train["label"].to_numpy(dtype=np.compat.long)
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)

<div dir="rtl">
<p>در این خانه test dataframe را به numpy ndarray و سپس به torch tensor dataset و در انتها به torch data loader تبدیل کردم.</p>
</div>

In [42]:
X_test = df_test.drop("label", axis=1).to_numpy(dtype=np.float32)
y_test = df_test["label"].to_numpy(dtype=np.compat.long)
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=True)

<div dir="rtl">
<p>در خانه زیر مدل را طراحی کرده ام.</p>
<p>در تابع initializer ورودی را ابتدا flatten می کنم و سپس از دو لایه perceptron که مقدار net آن ها از تابع relu عبور می کند، می گذرانم.</p>
<p>تعداد خروجی های هر لایه نمایانگر تعداد neuron های آن لایه می باشد.</p>
</div>

In [33]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

<div dir="rtl">
<p>یک instance از کلاس MLP که همان مدل من است می سازم، برای پردازش آن از device استفاده می کنم.</p>
</div>

In [34]:
MLP_model = MLP().to(device)
print(MLP_model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [35]:
for p in MLP_model.parameters():
    print(p.size())

torch.Size([512, 784])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([10, 512])
torch.Size([10])


<div dir="rtl">
<p>در اینجا توابع محاسبه خطا و بهینه ساز را تعریف کرده ام.</p>
</div>

In [36]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
optimizer = torch.optim.SGD(MLP_model.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adam(MLP_model.parameters(), lr=learning_rate)
# softmax = nn.Softmax(dim=1)

<div dir="rtl">
<p>در خانه زیر مدل را train کرده ام.</p>
<p>در هر epoch ابتدا مقدا gradient را برابر صفر قرار می دهیم.</p>
<p>سپس خروجی مدل را برای هر ورودی می یابیم.</p>
<p>مقدار loss را محاسبه می کنیم.</p>
<p>تغییرات وزن ها را با توجه به ورودی و loss را به کمک الگوریتم back propagation محاسبه می کنیم.</p>
<p>وزن ها را بروز میکنیم.</p>
<p>بعد از اتمام هر mini-batch مقدار loss را نمایش می دهیم.</p>
</div>

In [37]:
MLP_model.train()

for epoch in range(1, num_epochs+1):

    correct = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        # Zero gradient buffers
        optimizer.zero_grad()

        # Pass data through the network
        output = MLP_model(data)
#         output = softmax(output)
        
        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()

        # correct += (output == target).float().sum()

        # train_acc = torch.sum(output == target)
    # correct = (output == target).float().sum()
    print(f"Epoch: {epoch}; Loss: {loss.data}")

Epoch: 1; Loss: 0.1733751744031906
Epoch: 2; Loss: 0.16194002330303192
Epoch: 3; Loss: 0.09128453582525253
Epoch: 4; Loss: 0.18929554522037506
Epoch: 5; Loss: 0.031688936054706573
Epoch: 6; Loss: 0.023960717022418976
Epoch: 7; Loss: 0.04895363748073578
Epoch: 8; Loss: 0.022562455385923386
Epoch: 9; Loss: 0.023496493697166443
Epoch: 10; Loss: 0.02484445832669735


<div dir="rtl">
<p>در خانه زیر مدل را به کمک داده های test ارزیابی می کنیم.</p>
<p>در انتهای ارزیابی مقدار loss و accuracy را نمایش می دهیم.</p>
</div>

In [50]:
MLP_model.eval()
val_loss, correct = 0, 0
for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    output = MLP_model(data)
    val_loss += criterion(output, target).data.item()
    pred = output.data.max(1)[1] # get the index of the max log-probability
    correct += pred.eq(target.data).cpu().sum()

val_loss /= len(test_loader)

accuracy = 100. * correct.to(torch.float32) / len(test_loader.dataset)

print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    val_loss, correct, len(test_loader.dataset), accuracy))


Validation set: Average loss: 0.1126, Accuracy: 9675/10000 (97%)

